In [13]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from datetime import datetime
import time
import logging

In [33]:
sp500 = pd.read_html("SP500.html")[0]
sp500.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [35]:
sp500.tail()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
502,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [37]:
sp500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [64]:
sp500["Symbol"]

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

In [72]:
tickers = sp500["Symbol"].str.replace(".", "-", regex=False)

tickers.to_csv("SP500.csv", index=False) 

In [82]:
tickers = pd.read_csv("SP500.csv")
tickers.head()

,Symbol
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN


In [104]:
tickers = pd.read_csv("SP500.csv")["Symbol"].tolist()
logging.info(f"Loaded {len(tickers)} tickers ")

2025-12-15 22:54:49,833 - INFO - Loaded 503 tickers 


In [106]:
START_DATE = "2015-01-01"
END_DATE = datetime.today().strftime("%Y-%m-%d")
MIN_HISTORY_DAYS = 252 * 2

In [143]:
def download_price_data(
    tickers,
    start_date,
    end_date,
    pause=1.0
):
    all_prices = []

    for ticker in tickers:
        try:
            logging.info(f"Downloading {ticker}")

            data = yf.download(
                ticker,
                start=start_date,
                end=end_date,
                progress=False,
                auto_adjust=False
            )

            if data.empty:
                logging.warning(f"No data for {ticker}")
                continue

            data = data[["Adj Close"]].copy()
            data.columns = ["adj_close"]
            data["ticker"] = ticker
            data.reset_index(inplace=True)

            all_prices.append(data)
            time.sleep(pause)

        except Exception as e:
            logging.error(f"Failed for {ticker}: {e}")

    return pd.concat(all_prices, ignore_index=True)

In [149]:
price_df = download_price_data(
    tickers = tickers,
    start_date = START_DATE,
    end_date = END_DATE
)

logging.info(f"Downloaded {price_df['ticker'].nunique()} stocks")

2025-12-17 11:09:50,141 - INFO - Downloading MMM
2025-12-17 11:09:51,764 - INFO - Downloading AOS
2025-12-17 11:09:53,439 - INFO - Downloading ABT
2025-12-17 11:09:55,069 - INFO - Downloading ABBV
2025-12-17 11:09:56,643 - INFO - Downloading ACN
2025-12-17 11:09:58,258 - INFO - Downloading ADBE
2025-12-17 11:09:59,835 - INFO - Downloading AMD
2025-12-17 11:10:01,341 - INFO - Downloading AES
2025-12-17 11:10:02,910 - INFO - Downloading AFL
2025-12-17 11:10:04,614 - INFO - Downloading A
2025-12-17 11:10:06,207 - INFO - Downloading APD
2025-12-17 11:10:07,805 - INFO - Downloading ABNB
2025-12-17 11:10:09,221 - INFO - Downloading AKAM
2025-12-17 11:10:10,799 - INFO - Downloading ALB
2025-12-17 11:10:12,430 - INFO - Downloading ARE
2025-12-17 11:10:14,018 - INFO - Downloading ALGN
2025-12-17 11:10:15,678 - INFO - Downloading ALLE
2025-12-17 11:10:17,203 - INFO - Downloading LNT
2025-12-17 11:10:18,880 - INFO - Downloading ALL
2025-12-17 11:10:21,505 - INFO - Downloading GOOGL
2025-12-17 11:

In [190]:
price_df.head()

,Date,adj_close,ticker
0,2015-01-02,96.108559,MMM
1,2015-01-05,93.941040,MMM
2,2015-01-06,92.939323,MMM
3,2015-01-07,93.612999,MMM
4,2015-01-08,95.856659,MMM


In [192]:
price_df.tail()

,Date,adj_close,ticker
1333570,2025-12-08,118.160004,ZTS
1333571,2025-12-09,115.750000,ZTS
1333572,2025-12-10,118.070000,ZTS
1333573,2025-12-11,117.660004,ZTS
1333574,2025-12-12,118.660004,ZTS


In [194]:
price_df

,Date,adj_close,ticker
0,2015-01-02,96.108559,MMM
1,2015-01-05,93.941040,MMM
2,2015-01-06,92.939323,MMM
3,2015-01-07,93.612999,MMM
4,2015-01-08,95.856659,MMM
...,...,...,...
1333570,2025-12-08,118.160004,ZTS
1333571,2025-12-09,115.750000,ZTS
1333572,2025-12-10,118.070000,ZTS
1333573,2025-12-11,117.660004,ZTS


In [196]:
price_df = price_df.sort_values(["ticker", "Date"]).reset_index(drop=True)
price_df["Date"] = pd.to_datetime(price_df["Date"])

In [198]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333575 entries, 0 to 1333574
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   Date       1333575 non-null  datetime64[ns]
 1   adj_close  1333575 non-null  float64       
 2   ticker     1333575 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 30.5+ MB


In [202]:
price_df.isnull().value_counts()

Date   adj_close  ticker
False  False      False     1333575
Name: count, dtype: int64

Checking for tickers with short history!

In [227]:
obsv_per_ticker = (

    price_df.groupby("ticker").size().sort_values()
    
)
obsv_per_ticker.head()

ticker
Q        34
SOLS     39
SNDK    210
GEV     431
SOLV    432
dtype: int64

Date coverage analysis

In [234]:
date_coverage = (
    price_df
    .groupby("ticker")["Date"]
    .agg(start_date="min", end_date="max", observations="count")
    .sort_values("start_date")
)

date_coverage.head()

,start_date,end_date,observations
ticker,,,
A,2015-01-02,2025-12-11,2753
NRG,2015-01-02,2025-12-12,2754
NOW,2015-01-02,2025-12-12,2754
NOC,2015-01-02,2025-12-12,2754
NKE,2015-01-02,2025-12-12,2754


Adding daily returns

In [237]:
price_df.head()

,Date,adj_close,ticker
0,2015-01-02,37.120949,A
1,2015-01-05,36.425392,A
2,2015-01-06,35.857967,A
3,2015-01-07,36.333862,A
4,2015-01-08,37.422970,A


In [239]:
price_df["daily_return"] = (
    price_df
    .groupby("ticker")["adj_close"]
    .pct_change()
)

In [241]:
price_df.head()

,Date,adj_close,ticker,daily_return
0,2015-01-02,37.120949,A,NaN
1,2015-01-05,36.425392,A,-0.018738
2,2015-01-06,35.857967,A,-0.015578
3,2015-01-07,36.333862,A,0.013272
4,2015-01-08,37.422970,A,0.029975


In [243]:
price_df["daily_return"].describe(
    percentiles = [0.01, 0.05, 0.95, 0.99]
)

count    1.333073e+06
mean     6.585562e-04
std      2.073008e-02
min     -5.386473e-01
1%      -5.633841e-02
5%      -2.900372e-02
50%      7.779037e-04
95%      2.931318e-02
99%      5.755348e-02
max      7.459322e-01
Name: daily_return, dtype: float64

Volatility analysis by ticker

In [246]:
volatility_by_ticker = (
    price_df
    .groupby("ticker")["daily_return"]
    .std()
    .sort_values(ascending=False)
)

volatility_by_ticker.head()

ticker
SNDK    0.061748
COIN    0.054398
APP     0.048565
HOOD    0.046808
MRNA    0.045344
Name: daily_return, dtype: float64

Missing Data Behaviour

In [249]:
missing_returns = (
    price_df
    .groupby("ticker")["daily_return"]
    .apply(lambda x: x.isna().mean())
    .sort_values(ascending=False)
)

missing_returns.head()

ticker
Q       0.029412
SOLS    0.025641
SNDK    0.004762
GEV     0.002320
SOLV    0.002315
Name: daily_return, dtype: float64

Time Aggregation Check

In [252]:
monthly_prices = (
    price_df
    .set_index("Date")
    .groupby("ticker")["adj_close"]
    .resample("M")
    .last()
    .reset_index()
)

monthly_prices["monthly_return"] = (
    monthly_prices
    .groupby("ticker")["adj_close"]
    .pct_change()
)

/var/folders/kn/k8w5yjls0kxdc3_svrv6wy600000gn/T/ipykernel_64411/3739541240.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")


In [272]:
monthly_prices.head()

,ticker,Date,adj_close,monthly_return
0,A,2015-01-31,34.567509,NaN
1,A,2015-02-28,38.631039,0.117553
2,A,2015-03-31,38.120667,-0.013211
3,A,2015-04-30,37.955524,-0.004332
4,A,2015-05-31,37.790375,-0.004351


### Equal Weight Portfolio 

In [262]:
eligible_tickers = (
    price_df
    .groupby("ticker")
    .size()
    .loc[lambda x: x >= MIN_HISTORY_DAYS]
    .index
)

price_df_filtered = price_df[price_df["ticker"].isin(eligible_tickers)]

In [264]:
price_df_filtered.head()

,Date,adj_close,ticker,daily_return
0,2015-01-02,37.120949,A,NaN
1,2015-01-05,36.425392,A,-0.018738
2,2015-01-06,35.857967,A,-0.015578
3,2015-01-07,36.333862,A,0.013272
4,2015-01-08,37.422970,A,0.029975


In [312]:
portfolio_returns = (
    price_df_filtered
    .dropna(subset = ["daily_return"])
    .groupby("Date")["daily_return"]
    .mean()
    .rename("portfolio_return")
    .reset_index()
)

In [314]:
portfolio_returns["cumulative_return"] = (
    1 + portfolio_returns["portfolio_return"]
).cumprod()

In [316]:
portfolio_returns.head()

,Date,portfolio_return,cumulative_return
0,2015-01-05,-0.016382,0.983618
1,2015-01-06,-0.009250,0.974520
2,2015-01-07,0.012633,0.986831
3,2015-01-08,0.018306,1.004895
4,2015-01-09,-0.007566,0.997293


In [337]:
def rolling_cumulative_return(returns, window):
    return (1 + returns).rolling(window).apply(
        lambda x: x.prod() - 1, raw=False
    )
    

In [341]:
price_df["mom_1m"] = price_df.groupby("ticker")["daily_return"].transform(
    lambda x: rolling_cumulative_return(x, 21)
)

price_df["mom_3m"] = price_df.groupby("ticker")["daily_return"].transform(
    lambda x: rolling_cumulative_return(x, 63)
)

price_df["mom_6m"] = price_df.groupby("ticker")["daily_return"].transform(
    lambda x: rolling_cumulative_return(x, 126)
)

price_df["mom_3m"] = price_df.groupby("ticker")["daily_return"].transform(
    lambda x: rolling_cumulative_return(x, 252)
)

In [ ]:
for col in ["mom_1m", "mom_3m", "mom_6m", "mom_12m"]:
    price_df[f"{col}_pct"] = (
        price_df
        .groupby("Date")[col]
        .rank(pct=True)
    )

In [343]:
price_df.head()

,Date,adj_close,ticker,daily_return,mom_1m,mom_3m,mom_6m
0,2015-01-02,37.120949,A,NaN,NaN,NaN,NaN
1,2015-01-05,36.425392,A,-0.018738,NaN,NaN,NaN
2,2015-01-06,35.857967,A,-0.015578,NaN,NaN,NaN
3,2015-01-07,36.333862,A,0.013272,NaN,NaN,NaN
4,2015-01-08,37.422970,A,0.029975,NaN,NaN,NaN


In [345]:
price_df.tail()

,Date,adj_close,ticker,daily_return,mom_1m,mom_3m,mom_6m
1333570,2025-12-08,118.160004,ZTS,-0.004130,-0.011710,-0.317076,-0.277274
1333571,2025-12-09,115.750000,ZTS,-0.020396,-0.037342,-0.328900,-0.302565
1333572,2025-12-10,118.070000,ZTS,0.020043,-0.014358,-0.322005,-0.285293
1333573,2025-12-11,117.660004,ZTS,-0.003472,-0.020480,-0.330768,-0.298279
1333574,2025-12-12,118.660004,ZTS,0.008499,-0.021361,-0.319581,-0.272979
